# Leadership gap for women scientists in Switzerland

_work-in-progress_

[Joao Martins](https://github.com/zambujo)

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 6)})

# data
grants=pd.read_feather('data/project.feather')
grants=grants.dropna(subset=['start_date'])
grants['year'] = grants['start_date'].dt.year
grantees=pd.read_feather('data/grantee.feather')
people=pd.read_feather('data/person.feather')

# project leaders + project employees + gender + their projects
df=(grantees
      .query('role in ["employee", "responsible_applicant"]')
      .merge(people[['person_id_snsf', 'gender']], on='person_id_snsf')
      .merge(grants, on='project_number'))
# ...
df = df.drop(['index_x', 'index_y'], axis=1)
# to prevent error on pivoting
df['gender'] = df.gender.astype(str) # for pivoting later
df['role'] = df.role.astype(str) # for pivo

## The "leaky" pipeline

It is well-known that women are underrepresented in leadership positions.  Science is no exception. The "leaky" pipeline refers to the overly disproportionate loss of women throughout careers in science.

**If science careers are a "leaky" pipeline for women scientists, then the share of women leading science projects is expected to be lower than the share of women employed in science projects.**

---

See also: 
1. [Leaky pipeline for women scientists dries up after they win first big grant](https://www.nature.com/articles/d41586-018-05759-w), _in Nature, 2018_
2. [A Leak in the Academic Pipeline](https://www.frontiersin.org/articles/10.3389/fpsyg.2019.01297/full), _in Frontiers, 2019_

## Looking for evidence

The leading public science funding agency in Switzerland makes its data freely available:  
 - over 70k science projects, spanning across 50 years  
 - over 100k scientists  
 - function and gender of each scientist
 
Source: [p3.snf.ch](http://p3.snf.ch/Pages/DataAndDocumentation.aspx)

In [ ]:
# counts by year between 2000 and 2019
counts = (df
            .query('(year > 1999) & (year < 2020)')
            .groupby(['year', 'role', 'gender']).size()
            .reset_index().rename(columns={0:'count'}))
counts = counts.pivot_table(index=['year', 'role'], columns='gender', values='count', fill_value=0)
counts['both'] = (counts['male'] + counts['female'])
counts['percent_women'] = 100 * counts['female'] / counts['both']
counts['percent_men'] = 100 * counts['male'] / counts['both']

In [ ]:
counts.query('role=="employee"')

In [ ]:
sns.set_style("whitegrid")
plt.box(on=None)
g=sns.barplot(x='year', y='percent_women', color='#3f72af', data=counts.query('role == "employee"').reset_index())
g.set_xlabel('')
g.set_ylabel('')
g.set(ylim=(0, 100))
g.axhline(50, linestyle='--', alpha=.7)
g.set_title('Percentage of women employed in science projects', fontsize = 16, x=.28, y=1.05)
fig = g.get_figure()
fig.savefig("docs/img/plot1.svg")

## Share of women employed in science projects


![plot 1](./img/plot1.svg "Percentage of women employed in science projects")

The share of women employed in science projects raised from 38% in 2000 to 46% in 2019.

In [ ]:
counts.query('role=="responsible_applicant"')

In [ ]:
plt.box(on=None)
sns.set_style("whitegrid")
g=sns.barplot(x='year', y='percent_women', color='#3f72af', data=counts.query('role == "responsible_applicant"').reset_index())
g.set_xlabel('')
g.set_ylabel('')
g.set(ylim=(0, 100))
g.axhline(50, linestyle='--', alpha=.7)
g.set_title('Percentage of women leading science projects', fontsize = 16, x=.25, y=1.05)
fig = g.get_figure()
fig.savefig("docs/img/plot2.svg")

## Share of women leading science projects


![plot 2](./img/plot2.svg "Percentage of women leading science projects")

The share of women leading science projects raised from 15% in 2000 to 33% in 2019.

In [ ]:
gap_women = counts.pivot_table(index=['year'], columns='role', values='percent_women', fill_value=0)
gap_women['ratio'] = gap_women['responsible_applicant'] / gap_women['employee']
gap_men = counts.pivot_table(index=['year'], columns='role', values='percent_men', fill_value=0)
gap_men['ratio'] = gap_men['responsible_applicant'] / gap_men['employee']
gap = pd.concat([gap_women, gap_men], keys=['women', 'men'])
gap = gap.reset_index().rename(columns={'level_0':'gender'})

In [ ]:
# custom lineplot with matplotlib
mpl.rcParams['lines.linewidth']=3
plt.rcParams['figure.figsize']=(10, 6)
plt.gcf().set_size_inches(10, 6)
plt.style.use('seaborn-whitegrid')
plt.plot(gap.query('gender=="women"').year, 
         gap.query('gender=="women"').ratio,
         '-o',
         markersize=8,
         markerfacecolor='white',
         markeredgewidth=3)
plt.plot(gap.query('gender=="men"').year, 
         gap.query('gender=="men"').ratio,
         '-o',
         markersize=8,
         markerfacecolor='white',
         markeredgewidth=3)
plt.figtext(.85, .18, 
            u'Women', ha='left', va='bottom', 
            color='#1f77b4', fontsize='large', fontweight='semibold')
plt.figtext(.85, .6, 
            u'Men', ha='left', va='bottom', 
            color='#ff7f0e', fontsize='large', fontweight='semibold')
plt.axhline(y=1, color='black', linestyle='-', linewidth=3, alpha=.15)
plt.xlim(1998, 2021)
plt.ylim(.35, 1.55)
plt.title('Ratio between leaders and employees in science projects', fontsize='x-large', fontweight='light', x = .3)
plt.box(on=None)
plt.tight_layout()
plt.savefig("docs/img/plot3.svg")

## Leadership gap between men and women

![plot 3](./img/plot3.svg "Ratio bewteen people leading and employed in science projects")

The leadership gap between men and women appears to be steadily closing.

## Discussion

Steady decrease of the gap between the share of women leading science projects and the share of women employed in science projects.

Next steps: (1) domain-by-domain figures, (2) exclude one-person programmes.

---

Further reading:  
- [Promotion of women by the SNSF](http://www.snf.ch/en/researchinFocus/newsroom/Pages/news-181004-promotion-of-women-by-the-snsf.aspx)  
- [Fix the leaky pipeline](https://www.fix-the-leaky-pipeline.ch)
